In [56]:
%pip install ipykernel
%pip install tensorflow
%pip install numpy
%pip install keras-tuner
%pip install scikit-learn
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [57]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import Hyperband
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow.keras.backend as K
from keras_tuner import Objective



In [58]:
df = pd.read_csv('df_tratado.csv', sep=',')
df.head()

,Unnamed: 0,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,dia,mes,ano,grave
0,2,1032898.0,2007-08-13,Segunda,14:25:00,MG,40,585.5,ITABIRITO,Outras,...,0,1,2,0,1,1,13,8,2007,0
1,3,1051130.0,2007-02-12,Segunda,02:10:00,MA,135,11.0,SAO LUIS,Animais na Pista,...,2,1,0,0,3,1,12,2,2007,1
2,4,1066824.0,2007-11-20,terça,05:30:00,CE,222,30.8,CAUCAIA,Defeito mecânico em veículo,...,1,0,0,0,1,1,20,11,2007,0
3,5,1069918.0,2007-12-16,Domingo,17:40:00,MA,230,14.0,BARAO DE GRAJAU,Outras,...,0,1,0,0,1,1,16,12,2007,0
4,6,1070971.0,2007-03-05,Segunda,08:10:00,PR,277,584.4,CASCAVEL,Outras,...,0,0,2,0,0,2,5,3,2007,0


In [59]:
features = ['dia_semana','fase_dia','condicao_metereologica','tipo_pista','veiculos']
X = df[features]
y = df['grave']

In [60]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [61]:
X_encoded.head()

,veiculos,dia_semana_Quarta,dia_semana_Quinta,dia_semana_Segunda,dia_semana_Sexta,dia_semana_Sábado,dia_semana_Terça,dia_semana_quarta,dia_semana_quinta,dia_semana_sexta,dia_semana_terça,fase_dia_Anoitecer,fase_dia_Plena Noite,fase_dia_Plena noite,fase_dia_Pleno dia,condicao_metereologica_Neblina,condicao_metereologica_Sol,condicao_metereologica_neve,tipo_pista_Múltipla,tipo_pista_Simples
0,1,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True
1,1,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True
2,1,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
3,1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
4,2,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,True


In [62]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [64]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 7)):
        model.add(
            layers.Dense(
                units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
                activation=hp.Choice('activation', ['relu', 'tanh'])
            )
        )
    
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=[f1_score]
    )
    return model

In [65]:
def f1_score(y_true, y_pred):
    y_pred_bin = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred_bin, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_bin, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_bin), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

In [66]:
tuner = Hyperband(
    build_model,
    objective=Objective("val_f1_score", direction="max"),  # <- agora funciona!
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='modelo_acidente_f1'
)

In [67]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [68]:
tuner.search(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[stop_early])

Trial 90 Complete [00h 06m 47s]
val_f1_score: 0.0

Best val_f1_score So Far: 0.0006443727761507034
Total elapsed time: 04h 41m 33s


In [69]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("📊 Melhores hiperparâmetros encontrados:")
for param, value in best_hp.values.items():
    print(f"{param}: {value}")


📊 Melhores hiperparâmetros encontrados:
num_layers: 1
units_0: 112
activation: relu
lr: 0.002579876453443278
units_1: 64
units_2: 96
units_3: 112
units_4: 32
units_5: 80
units_6: 96
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0


In [70]:

history = best_model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test))

# ==== 6. Avaliar ====
loss, f1 = best_model.evaluate(X_test, y_test)
print(f'\n✅ F1-score no teste: {f1:.4f}')

Epoch 1/150
45005/45005 [==============================] - 39s 859us/step - loss: 0.1630 - f1_score: 2.5849e-04 - val_loss: 0.1631 - val_f1_score: 3.4068e-04
Epoch 2/150
45005/45005 [==============================] - 38s 851us/step - loss: 0.1629 - f1_score: 1.8887e-04 - val_loss: 0.1630 - val_f1_score: 8.8873e-05
Epoch 3/150
45005/45005 [==============================] - 39s 860us/step - loss: 0.1630 - f1_score: 1.9257e-04 - val_loss: 0.1632 - val_f1_score: 8.8873e-05
Epoch 4/150
45005/45005 [==============================] - 39s 861us/step - loss: 0.1629 - f1_score: 1.4147e-04 - val_loss: 0.1632 - val_f1_score: 0.0000e+00
Epoch 5/150
45005/45005 [==============================] - 38s 853us/step - loss: 0.1628 - f1_score: 1.7776e-04 - val_loss: 0.1629 - val_f1_score: 2.3699e-04
Epoch 6/150
45005/45005 [==============================] - 39s 857us/step - loss: 0.1628 - f1_score: 1.8887e-04 - val_loss: 0.1629 - val_f1_score: 1.4812e-04
Epoch 7/150
45005/45005 [===========================